In [14]:
import pandas as pd
import numpy as np
from datetime import datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX

import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

df = pd.read_parquet("cmm_erdos_bootcamp_2020_timeseries.pq", engine='pyarrow')
df.date_val = pd.to_datetime(df.date_val)

df['series'] = df['volume_C']
order=(0,1,1)
seasonal_order=(0,1,1,7)
exog_feat = ['is_holiday']

#print(order, seasonal_order, exog_feat)

train_test_sep = datetime(2019, 1, 1)
df_train = df.loc[df.date_val < train_test_sep]
df_test = df.loc[df.date_val >= train_test_sep]

train_ = df_train.series.values
test_ = df_test.series.values
#np.disp(len(test_))

exog_train = df_train[exog_feat]
exog_test = df_test[exog_feat]

model = SARIMAX(train_, order=order, seasonal_order=seasonal_order, exog=exog_train)
fit_ = model.fit(disp=False, method='powell')

forecast_ = fit_.get_forecast(len(test_), exog=exog_test)

np.disp(np.abs((test_ - forecast_.predicted_mean) / test_).mean())

data = pd.DataFrame(forecast_.conf_int())
data['mean'] = forecast_.predicted_mean
data = data[["mean","lower y","upper y"]]
data.to_csv('C_12m_sarimax_0110117_is_holiday.csv',index=False,header=False)

0.16386263403291743
